# Phonon computations

This is a quick sketch how to run a simple phonon calculation using DFTK.

> **Preliminary implementation**
>
> Practical phonon computations have only seen rudimentary testing as of now.
> As of now we do not yet recommend relying on this feature for production
> calculations. We appreciate any issues, bug reports or PRs.

First we run an SCF calculation.

In [1]:
using AtomsBuilder
using DFTK
using PseudoPotentialData

pseudopotentials = PseudoFamily("cp2k.nc.sr.lda.v0_1.semicore.gth")
model  = model_DFT(bulk(:Si); pseudopotentials, functionals=LDA())
basis  = PlaneWaveBasis(model; Ecut=10, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-8);
nothing  # hide

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime 
---   ---------------   ---------   ---------   ----   ------
  1   -7.916451415585                   -0.69    4.9    110ms
  2   -7.921213536530       -2.32       -1.52    1.0    102ms
  3   -7.921407007434       -3.71       -2.49    1.2   57.6ms
  4   -7.921440951105       -4.47       -2.83    2.5   75.7ms
  5   -7.921441676240       -6.14       -3.04    1.1   56.1ms
  6   -7.921442009010       -6.48       -4.54    1.1   56.2ms
  7   -7.921442021907       -7.89       -4.68    2.9   84.2ms
  8   -7.921442022132       -9.65       -5.35    1.0   54.9ms
  9   -7.921442022143      -10.96       -5.72    1.8   87.8ms
 10   -7.921442022144      -11.90       -6.13    1.0   55.9ms
 11   -7.921442022144      -12.81       -6.96    1.1    230ms
 12   -7.921442022144      -13.75       -7.88    2.2   73.3ms
 13   -7.921442022144   +    -Inf       -8.14    2.1   73.6ms


Next we compute the phonon modes at the q-point `[1/4, 1/4, 1/4]`.

In [2]:
scfres = DFTK.unfold_bz(scfres)
phret_q0 = @time DFTK.phonon_modes(scfres; q=[0.25, 0.25, 0.25]);
nothing  # hide

Iter  Restart  Krydim  log10(res)  avg(CG)  Δtime   Comment
----  -------  ------  ----------  -------  ------  ---------------
                                      73.0   2.09s  Non-interacting
   1        0       1        0.31     52.4   9.49s  
   2        0       2       -0.52     47.0   1.44s  
   3        0       3       -2.40     41.7   1.25s  
   4        0       4       -4.46     29.7   965ms  
   5        0       5       -6.46     17.4   711ms  
   6        0       6       -8.91      4.0   423ms  
   7        1       1       -7.82     55.7   1.99s  Restart
   8        1       2       -8.82      5.6   465ms  
                                      72.7   1.78s  Final orbitals
Iter  Restart  Krydim  log10(res)  avg(CG)  Δtime   Comment
----  -------  ------  ----------  -------  ------  ---------------
                                      72.9   1.60s  Non-interacting
   1        0       1        0.31     52.4   1.53s  
   2        0       2       -0.52     47.0   1.32s  
   3

These are the final phonon frequencies:

In [3]:
phret_q0.frequencies

6-element Vector{Float64}:
 -0.003498155355542753
 -0.00297745115932751
 -0.0029774511591944153
  0.004302230643459321
  0.00430223064398163
  0.004353201771994588